Questão 2
O ciclo de crédito do cartão Fortbrasil consiste em 3 etapas: Aquisição, Manutenção e Cobrança. 
O cliente entra em cobrança com 5 dias de atraso, e quando atinge a marca de 65 dias tem o cartão cancelado 
e então para realizar o pagamento da dívida precisa realizar um acordo. A FortBrasil irá fazer uma nova campanha 
para aumentar a adesão aos acordos e gostaria de traçar uma estratégia de cobrança utilizando as informações históricas 
e técnicas de modelagem preditivas. 

Segue abaixo as definições das variáveis:
    ID_CONTA: Identificador da conta;
    DT_ACORDO: Data do acordo;
    NU_DIAS_ATRASO: Número de dias após o vencimento que esse cliente está sem pagar;
    VALOR_CRELIQ: Valor da dívida do cliente quando ele ultrapassa os 65 dias de atraso;
    DIVIDA_ATUAL: Dívida atualizada com juros;
    RESPOSTA: Se for 1, significa que o cliente aderiu aquele acordo. Ou seja, efetuou o pagamento da entrada. Se for 0, caso contrário.
    QTD_PARCELAMENTO_XM: Quantidade de parcelamentos de fatura aderidos X meses antes da fatura que ele deixou de pagar.
    LIMITE: Limite total do cliente;
    QTD_EXTRATOS: Quantidade de Faturas emitidas;
    QTD_FX0: Quantidade de faturas emitidas em dia;
    QTD_FX1: Quantidade de faturas emitidas no qual o cliente não pagou a fatura do mês anterior;
    QTD_FX2: Quantidade de faturas emitidas no qual o cliente não pagou as duas ultimas faturas anteriores;
    QTD_ACIONAMENTO: Quantidade de vezes que ligamos para os clientes para cobrar a sua dívida;
    QTD_CP: Quantidade de vezes que a ligação foi atendida, mas não houve contato com o cliente;
    QTD_CPC: Quantidade de vezes que o próprio cliente atendeu a ligação de cobrança.
    DEFINIÇÃO: Acordo é uma proposta de negociação da dívida por parte da Fortbrasil para clientes que estão com mais de 180 dias de atraso.
    Baseado nessas informações, responda:


    2.2 – Qual modelo você utilizaria para traçar uma estratégia objetivando o aumento da adesão dos acordos? (Descreva a técnica utilizada)
    2.3 – Quais indicadores e ferramentas você utilizaria para avaliar a performance/aderência desse modelo? (Descreva os indicadores utilizados)
2.4 – Apresente o modelo desenvolvido utilizando a técnica do item (2.2) e as técnicas de avaliação descritas no item (2.3).


Importação das bibliotecas necessárias

In [1]:
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
path = 'Case Cientista JR/Questão 2(consolidada).csv'
adesao_acordos = pd.read_csv(path, sep=';' ,encoding='Windows-1252', decimal=',')
adesao_acordos.head()

,ID_CONTA,DT_ACORDO,NU_DIAS_ATRASO,VALOR_CRELIQ,DIVIDA_ATUAL,RESPOSTA,QTD_PARCELAMENTO_3M,QTD_PARCELAMENTO_6M,QTD_PARCELAMENTO_12M,LIMITE,...,QTD_CPC_6M,QTD_CP_10D,QTD_CP_1M,QTD_CP_3M,QTD_CP_6M,QTD_ACIONAMENTO_10D,QTD_ACIONAMENTO_1M,QTD_ACIONAMENTO_3M,QTD_ACIONAMENTO_6M,NU_DIAS_ATRASO_GROUP
0,1194908,2018-11-22,221,324.38,463.73,0,0,0,0,1820.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,>6M
1,1447997,2018-11-22,191,334.82,451.56,1,0,0,0,449.0,...,4.0,0.0,1.0,2.0,2.0,8.0,11.0,78.0,241.0,>6M
2,1035415,2018-11-22,252,1682.26,2554.18,0,0,0,0,1208.0,...,0.0,0.0,0.0,0.0,0.0,2.0,3.0,6.0,16.0,>8M
3,1069434,2018-11-22,231,1625.01,2368.65,0,0,0,0,3499.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,>6M
4,1427566,2018-11-22,270,320.07,503.25,0,0,0,0,270.0,...,0.0,0.0,1.0,1.0,1.0,10.0,13.0,13.0,13.0,>8M


In [3]:
#adesao_acordos = adesao_acordos[["ID_CONTA","NU_DIAS_ATRASO", "DIVIDA_ATUAL", "QTD_PARCELAMENTO_6M", "QTD_FX0_GERAL","QTD_ACIONAMENTO_6M","QTD_CPC_6M"]]
adesao_acordos = adesao_acordos[["ID_CONTA","NU_DIAS_ATRASO","QTD_ACIONAMENTO_6M","QTD_CPC_6M","RESPOSTA"]]
adesao_acordos.head()


,ID_CONTA,NU_DIAS_ATRASO,QTD_ACIONAMENTO_6M,QTD_CPC_6M,RESPOSTA
0,1194908,221,36.0,0.0,0
1,1447997,191,241.0,4.0,1
2,1035415,252,16.0,0.0,0
3,1069434,231,1.0,0.0,0
4,1427566,270,13.0,0.0,0


In [4]:
adesao_acordos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72532 entries, 0 to 72531
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_CONTA            72532 non-null  int64  
 1   NU_DIAS_ATRASO      72532 non-null  int64  
 2   QTD_ACIONAMENTO_6M  70462 non-null  float64
 3   QTD_CPC_6M          70462 non-null  float64
 4   RESPOSTA            72532 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 2.8 MB


In [5]:
#Converte a tipagem das features
adesao_acordos['ID_CONTA'] = adesao_acordos['ID_CONTA'].astype(int).astype(str)
#adesao_acordos[['QTD_ACIONAMENTO_6M','QTD_CPC_6M']] = adesao_acordos[['QTD_ACIONAMENTO_6M','QTD_CPC_6M']].astype(float).astype(int)
adesao_acordos[['QTD_ACIONAMENTO_6M','QTD_CPC_6M']] = adesao_acordos[['QTD_ACIONAMENTO_6M','QTD_CPC_6M']].fillna(0.0).astype(int)
adesao_acordos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72532 entries, 0 to 72531
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID_CONTA            72532 non-null  object
 1   NU_DIAS_ATRASO      72532 non-null  int64 
 2   QTD_ACIONAMENTO_6M  72532 non-null  int32 
 3   QTD_CPC_6M          72532 non-null  int32 
 4   RESPOSTA            72532 non-null  int64 
dtypes: int32(2), int64(2), object(1)
memory usage: 2.2+ MB


In [6]:
# descartar (Linhas axis=0)|(colunas axis=1) nas quais mais de 10% dos valores estão ausentes
#adesao_acordos_teste = adesao_acordos.dropna(thresh=len(adesao_acordos)*0.9, axis=0)
adesao_acordos = adesao_acordos.dropna(axis = 0)
adesao_acordos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72532 entries, 0 to 72531
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID_CONTA            72532 non-null  object
 1   NU_DIAS_ATRASO      72532 non-null  int64 
 2   QTD_ACIONAMENTO_6M  72532 non-null  int32 
 3   QTD_CPC_6M          72532 non-null  int32 
 4   RESPOSTA            72532 non-null  int64 
dtypes: int32(2), int64(2), object(1)
memory usage: 2.8+ MB


In [7]:
adesao_acordos.head()

,ID_CONTA,NU_DIAS_ATRASO,QTD_ACIONAMENTO_6M,QTD_CPC_6M,RESPOSTA
0,1194908,221,36,0,0
1,1447997,191,241,4,1
2,1035415,252,16,0,0
3,1069434,231,1,0,0
4,1427566,270,13,0,0


In [8]:
X = adesao_acordos[["NU_DIAS_ATRASO","QTD_ACIONAMENTO_6M","QTD_CPC_6M"]].values
y = adesao_acordos["RESPOSTA"].values

Passo 2 - Separar dados de treino e dados de teste

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 )#,train_size=0.4

Passo 3 - Treinamos nossa Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

random_forest_clf = RandomForestClassifier()
random_forest_clf.fit(X_train, y_train)

y_pred = random_forest_clf.predict(X_test)

Passo 4 - Verificamos a matriz de confusão e as métricas do nosso modelo


In [17]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
#returned = confusion_matrix(y_test, y_pred).ravel()
#if len(returned) == 4:
#    tn, fp, fn, tp = returned

print(tp, fp, tn, fn)
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

4 111 13968 424
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     14079
           1       0.03      0.01      0.01       428

    accuracy                           0.96     14507
   macro avg       0.50      0.50      0.50     14507
weighted avg       0.94      0.96      0.95     14507

0.9631212518094713
